In [1]:

from googleapiclient.discovery import build
import pandas as pd
from datetime import datetime

# Configuration
API_KEY = "API_Key"
TARGET_VIDEOS = 100  # Number of trending videos to fetch
REGION_CODE = None # Using for Global content

# Initialize YouTube API client
youtube = build("youtube", "v3", developerKey=API_KEY)

def get_subscribers(channel_id):
    """Fetch subscriber count for a given channel ID."""
    try:
        request = youtube.channels().list(part="statistics", id=channel_id)
        response = request.execute()
        return int(response["items"][0]["statistics"]["subscriberCount"]) if response["items"] else 0
    except Exception as e:
        print(f"Error fetching subscribers for channel {channel_id}: {e}")
        return 0

def fetch_trending_videos():
    """Fetch top 100 trending videos with details."""
    videos_data = []
    max_results_per_call = 50  # API max per call
    page_token = None

    try:
        while len(videos_data) < TARGET_VIDEOS:
            request = youtube.videos().list(
                part="snippet,statistics,contentDetails",
                chart="mostPopular",
                regionCode=REGION_CODE,
                maxResults=max_results_per_call,
                pageToken=page_token
            )
            response = request.execute()
            print(f"Fetched {len(response['items'])} videos in this batch")

            for video in response["items"]:
                videos_data.append({
                    "Video Name": video["snippet"]["title"],
                    "Views": int(video["statistics"]["viewCount"]),
                    "Likes": int(video["statistics"].get("likeCount", 0)),
                    "Channel Name": video["snippet"]["channelTitle"],
                    "Post Date": video["snippet"]["publishedAt"],
                    "Duration": video["contentDetails"]["duration"],
                    "Subscribers": get_subscribers(video["snippet"]["channelId"]),
                    "Description": video["snippet"]["description"],
                    "Category": video["snippet"]["categoryId"]
                })

            page_token = response.get("nextPageToken")
            if not page_token or len(videos_data) >= TARGET_VIDEOS:
                break

        return videos_data[:TARGET_VIDEOS]  # Trim to exactly 100

    except Exception as e:
        print(f"Error fetching data: {e}")
        return []

# Fetch and process data
videos_data = fetch_trending_videos()

# Display sample output
if videos_data:
    print("\nSample of Top 5 Trending Videos:")
    for video in videos_data[:5]:
        print(f"Title: {video['Video Name']}, Channel: {video['Channel Name']}, Views: {video['Views']:,}")

    # Save to Excel
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"trending_videos_{timestamp}.xlsx"
    df = pd.DataFrame(videos_data)
    df.to_excel(filename, index=False)
    print(f"\nData saved to {filename} with {len(videos_data)} rows")
else:
    print("No data fetched. Check API key or network.")

Fetched 50 videos in this batch
Fetched 50 videos in this batch

Sample of Top 5 Trending Videos:
Title: Jack Black - Steve's Lava Chicken (Official Music Video) | A Minecraft Movie Soundtrack | WaterTower, Channel: WaterTower Music, Views: 604,413
Title: Murderbot — Official Trailer | Apple TV+, Channel: Apple TV, Views: 1,320,684
Title: Mario Kart Should Cost More, Channel: videogamedunkey, Views: 1,328,670
Title: He was Attacking Everyone, Then I Found Out Why 🥺, Channel: Rocky Kanaka, Views: 508,268
Title: HIGHLIGHTS - Arsenal vs Real Madrid | UEFA Champions League - 4tos Final 24/25 | TUDN, Channel: TUDN USA, Views: 1,087,945

Data saved to trending_videos_20250410_144053.xlsx with 100 rows


In [2]:
# Fetch category names
def get_category_mapping():
    """Fetch mapping of category IDs to names."""
    try:
        request = youtube.videoCategories().list(
            part="snippet",
            regionCode="US"
        )
        response = request.execute()
        return {item["id"]: item["snippet"]["title"] for item in response["items"]}
    except Exception as e:
        print(f"Error fetching categories: {e}")
        return {}

category_mapping = get_category_mapping()

In [5]:
!pip install matplotlib


  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   ------------------ --------------------- 3.7/8.1 MB 19.8 MB/s eta 0:00:01
   ------------------------------------- -- 7.6/8.1 MB 19.7 MB/s eta 0:00:01
   ---------------------------------------- 8.1/8.1 MB 18.8 MB/s eta 0:00:00
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---------------------------------------- 2.2/2.2 MB 18.0 MB/s eta 0:00:00


In [6]:
# Analysis and dashboard

import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt

category_mapping = get_category_mapping()
df = pd.read_excel("trending_videos_20250410_144053.xlsx")
df["Category Name"] = df["Category"].astype(str).map(category_mapping)

top_categories = df["Category Name"].value_counts().head(5)
print("\nTop 5 Video Categories:")
print(top_categories)

avg_views_by_category = df.groupby("Category Name")["Views"].mean().sort_values(ascending=False).head(5)
print("\nAverage Views by Category (Top 5):")
print(avg_views_by_category)

top_channels = df[["Channel Name", "Subscribers"]].drop_duplicates().sort_values(by="Subscribers", ascending=False).head(5)
print("\nTop 5 Channels by Subscribers:")
print(top_channels)

plt.figure(figsize=(10, 6))
plt.bar(top_categories.index, top_categories.values, color="skyblue")
plt.xlabel("Category")
plt.ylabel("Number of Videos")
plt.title("Top 5 Trending Video Categories")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.savefig("top_categories.png")
plt.close()

plt.figure(figsize=(10, 6))
plt.bar(top_channels["Channel Name"], top_channels["Subscribers"], color="lightgreen")
plt.xlabel("Channel Name")
plt.ylabel("Subscribers")
plt.title("Top 5 Channels by Subscribers")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.savefig("top_channels.png")
plt.close()


Top 5 Video Categories:
Category Name
Sports            27
Entertainment     25
People & Blogs    10
Gaming             9
Music              7
Name: count, dtype: int64

Average Views by Category (Top 5):
Category Name
Film & Animation    1.154648e+07
People & Blogs      4.658089e+06
Gaming              2.446957e+06
Entertainment       2.307104e+06
News & Politics     2.177418e+06
Name: Views, dtype: float64

Top 5 Channels by Subscribers:
    Channel Name  Subscribers
82    Markiplier     37400000
57     LazarBeam     22900000
13           NBA     22900000
88       Sidemen     22300000
95  CoryxKenshin     21600000


In [7]:
# Average views by category (Horizontal Bar Chart)
plt.figure(figsize=(10, 6))
plt.barh(avg_views_by_category.index, avg_views_by_category.values, color="salmon")
plt.xlabel("Average Views")
plt.ylabel("Category")
plt.title("Average Views by Top 5 Categories")
plt.tight_layout()
plt.savefig("avg_views_by_category.png")
plt.close()

# Subscribers of top 5 channels (Pie Chart)
plt.figure(figsize=(8, 8))
plt.pie(top_channels["Subscribers"], labels=top_channels["Channel Name"], autopct="%1.1f%%", colors=["#FF9999", "#66B2FF", "#99FF99", "#FFCC99", "#FFD700"])
plt.title("Subscribers of Top 5 Channels")
plt.tight_layout()
plt.savefig("top_channels_subscribers.png")
plt.close()